In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv
/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv
/kaggle/input/equity-post-HCT-survival-predictions/train.csv
/kaggle/input/equity-post-HCT-survival-predictions/test.csv


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
train_df = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
test_df = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

In [4]:
train_df["age_comorbidity"] = train_df["age_at_hct"] * train_df["comorbidity_score"]
test_df["age_comorbidity"] = test_df["age_at_hct"] * test_df["comorbidity_score"]
columns_to_drop = ["efs", "efs_time", "ID", "cyto_score_detail", "gvhd_proph"]

In [5]:
X_train = train_df.drop(columns=columns_to_drop, axis=1)
y_train = train_df["efs_time"].values

In [6]:
num_features = X_train.select_dtypes(include=["float64", "int64"]).columns.tolist()
cat_features = X_train.select_dtypes(include=["object"]).columns.tolist()

In [7]:
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_features),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="Missing")),
        ("encoder", OneHotEncoder(drop="first", min_frequency=0.05, sparse=False))
    ]), cat_features)
])

In [8]:
X_train_processed = preprocessor.fit_transform(X_train).astype(np.float32)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_processed, y_train, test_size=0.2, random_state=42)

In [10]:
dtrain = xgb.DMatrix(X_tr, label=y_tr)
dval = xgb.DMatrix(X_val, label=y_val)

In [11]:
params = {
    "objective": "survival:cox",    # Cox proportional hazards model
    "eval_metric": "cox-nloglik",   # Negative log partial likelihood
    "max_depth": 4,
    "subsample": 0.7,
    "tree_method": "hist",
    "verbosity": 0,
}

In [12]:
evals = [(dtrain, "train"), (dval, "eval")]
bst = xgb.train(params, dtrain, num_boost_round=100, evals=evals, early_stopping_rounds=10)

[0]	train-cox-nloglik:9.01255	eval-cox-nloglik:7.62624
[1]	train-cox-nloglik:8.99312	eval-cox-nloglik:7.60861
[2]	train-cox-nloglik:8.97975	eval-cox-nloglik:7.59617
[3]	train-cox-nloglik:8.97132	eval-cox-nloglik:7.58940
[4]	train-cox-nloglik:8.96428	eval-cox-nloglik:7.58425
[5]	train-cox-nloglik:8.95839	eval-cox-nloglik:7.58030
[6]	train-cox-nloglik:8.95482	eval-cox-nloglik:7.57600
[7]	train-cox-nloglik:8.95111	eval-cox-nloglik:7.57500
[8]	train-cox-nloglik:8.94777	eval-cox-nloglik:7.57268
[9]	train-cox-nloglik:8.94491	eval-cox-nloglik:7.57114
[10]	train-cox-nloglik:8.94243	eval-cox-nloglik:7.57049
[11]	train-cox-nloglik:8.94000	eval-cox-nloglik:7.57296
[12]	train-cox-nloglik:8.93759	eval-cox-nloglik:7.57318
[13]	train-cox-nloglik:8.93558	eval-cox-nloglik:7.57533
[14]	train-cox-nloglik:8.93393	eval-cox-nloglik:7.57487
[15]	train-cox-nloglik:8.93258	eval-cox-nloglik:7.57524
[16]	train-cox-nloglik:8.93080	eval-cox-nloglik:7.57503
[17]	train-cox-nloglik:8.92880	eval-cox-nloglik:7.57323
[1

In [13]:
X_test = test_df.drop(columns=["ID", "cyto_score_detail", "gvhd_proph"], axis=1)
X_test_processed = preprocessor.transform(X_test).astype(np.float32)

In [14]:
dtest = xgb.DMatrix(X_test_processed)
risk_scores = bst.predict(dtest)

In [15]:
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "prediction": risk_scores
})

In [16]:
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

Submission file saved as submission.csv
